## Load the data set and make a copy

In [5]:
import pandas as pd
import os

In [6]:
cwd       = os.getcwd()
file_path = r"Data\dataset.csv"

dataset   = os.path.join(cwd,file_path)
df_original = pd.read_csv(dataset)
df_original['TIME'] = pd.to_datetime(df_original['TIME'], errors='coerce')

print("Data set Loaded successfully")

Data set Loaded successfully


In [7]:
df = df_original.copy()

## Taking a look at the dataset

In [8]:
df.head()

,STATION ID,TIME,LAST UPDATED,NAME,BIKE_STANDS,AVAILABLE_BIKE_STANDS,AVAILABLE_BIKES,STATUS,ADDRESS,LATITUDE,LONGITUDE
0,1,2023-06-01 00:00:03,2023-05-31 23:59:30,CLARENDON ROW,31,31,0,OPEN,Clarendon Row,53.3409,-6.26250
1,2,2023-06-01 00:00:03,2023-05-31 23:59:14,BLESSINGTON STREET,20,10,10,OPEN,Blessington Street,53.3568,-6.26814
2,3,2023-06-01 00:00:03,2023-05-31 23:58:39,BOLTON STREET,20,14,6,OPEN,Bolton Street,53.3512,-6.26986
3,4,2023-06-01 00:00:03,2023-05-31 23:50:47,GREEK STREET,20,6,14,OPEN,Greek Street,53.3469,-6.27298
4,5,2023-06-01 00:00:03,2023-05-31 23:56:14,CHARLEMONT PLACE,40,6,34,OPEN,Charlemont Street,53.3307,-6.26018


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163590 entries, 0 to 163589
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   STATION ID             163590 non-null  int64         
 1   TIME                   163590 non-null  datetime64[ns]
 2   LAST UPDATED           163590 non-null  object        
 3   NAME                   163590 non-null  object        
 4   BIKE_STANDS            163590 non-null  int64         
 5   AVAILABLE_BIKE_STANDS  163590 non-null  int64         
 6   AVAILABLE_BIKES        163590 non-null  int64         
 7   STATUS                 163590 non-null  object        
 8   ADDRESS                163590 non-null  object        
 9   LATITUDE               163590 non-null  float64       
 10  LONGITUDE              163590 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(4), object(4)
memory usage: 13.7+ MB


In [10]:
df.tail()

,STATION ID,TIME,LAST UPDATED,NAME,BIKE_STANDS,AVAILABLE_BIKE_STANDS,AVAILABLE_BIKES,STATUS,ADDRESS,LATITUDE,LONGITUDE
163585,113,2023-06-30 23:30:03,2023-06-30 23:27:34,MERRION SQUARE SOUTH,40,35,3,OPEN,Merrion Square South,53.3386,-6.24861
163586,114,2023-06-30 23:30:03,2023-06-30 23:20:11,WILTON TERRACE (PARK),40,38,2,OPEN,Wilton Terrace (Park),53.3337,-6.24834
163587,115,2023-06-30 23:30:03,2023-06-30 23:21:31,KILLARNEY STREET,30,16,13,OPEN,Killarney Street,53.3548,-6.24758
163588,116,2023-06-30 23:30:03,2023-06-30 23:25:42,BROADSTONE,30,28,2,OPEN,Broadstone,53.3547,-6.27231
163589,117,2023-06-30 23:30:03,2023-06-30 23:27:01,HANOVER QUAY EAST,40,40,0,OPEN,Hanover Quay East,53.3437,-6.23175


In [11]:
df2 = df.drop(['STATION ID','TIME','LONGITUDE','LATITUDE'], axis=1)
df2.describe().round(2)

,BIKE_STANDS,AVAILABLE_BIKE_STANDS,AVAILABLE_BIKES
count,163590.00,163590.00,163590.00
mean,31.98,20.22,11.35
std,7.46,11.51,10.01
min,16.00,0.00,0.00
25%,29.00,11.00,3.00
50%,30.00,20.00,9.00
75%,40.00,29.00,18.00
max,40.00,40.00,40.00


## Handling missing values

- As we have a humongous dataset and we need it for training. So its ok to drop a few rows to handle the missing values.
- If we had many missing values, we could've used imputers

In [12]:
if df.isnull().values.any():
    print("Has null")
    display(df.isnull().sum())

In [13]:
df.dropna(inplace=True)

df.isnull().sum()

STATION ID               0
TIME                     0
LAST UPDATED             0
NAME                     0
BIKE_STANDS              0
AVAILABLE_BIKE_STANDS    0
AVAILABLE_BIKES          0
STATUS                   0
ADDRESS                  0
LATITUDE                 0
LONGITUDE                0
dtype: int64

## Cleaning datasets

- Rule 1: Negative bikes
- Rule 2: Bikes > Total Capacity
- Rule 3: Available stands > Total Capacity

If any of these are found, we are removing them. As these should be some sensor error and can create some unnessary bias to the model

Check for duplicates and drop if any, as they become redundant rows for model training


In [14]:
df.duplicated().value_counts()

False    163590
Name: count, dtype: int64

In [15]:
## Rule 1

display((df['AVAILABLE_BIKES'] < 0).value_counts(normalize=True))
df = df[df['AVAILABLE_BIKES'] >= 0]

AVAILABLE_BIKES
False    1.0
Name: proportion, dtype: float64

In [16]:
# Rule 2

display((df['AVAILABLE_BIKES'] > df['BIKE_STANDS']).value_counts(normalize=True))

df = df[df['AVAILABLE_BIKES'] <= df['BIKE_STANDS']]

False    1.0
Name: proportion, dtype: float64

In [17]:
# Rule 3    

display((df['AVAILABLE_BIKE_STANDS'] > df['BIKE_STANDS']).value_counts(normalize=True))

df = df[df['AVAILABLE_BIKE_STANDS'] <= df['BIKE_STANDS']]

False    1.0
Name: proportion, dtype: float64

In [18]:
df.to_csv('Data/dataset_cleaned.csv',index=False)
print("Cleaned Data and Saved to 'Data/' directory")

Cleaned Data and Saved to 'Data/' directory
